In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Install allennlp(only need to run one time)

In [ ]:
!pip install allennlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 25.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.8 MB/s eta 0:00:00
 

In [ ]:
!pip install allennlp_models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.5/464.5 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.7 MB/s eta 0:00:00
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
INFO: pip is looking at multiple versions of cached-path to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 16.8 MB/s eta 0:00:00
INFO

### Import & Data Loading

In [2]:
#import allennlp_models.tagging
import spacy
import pandas as pd
from spacy import displacy
from spacy import tokenizer
from collections import Counter
#from allennlp.predictors.predictor import Predictor
from statsmodels.stats.proportion import proportions_ztest
nlp = spacy.load('en_core_web_sm')
nlp.max_length = 1200000

### Load text after Coreference

In [ ]:
#predictor_cor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz")

In [ ]:
dir = 'gdrive/My Drive/1012 Project/'
with open(dir+'JE_coref.txt') as f:
  JE_coref = f.read()
with open(dir+'LO_coref.txt') as f:
  LO_coref = f.read()
with open(dir+'PP_coref.txt') as f:
  PP_coref = f.read()
with open(dir+'GE_coref.txt') as f:
  GE_coref = f.read()

## replace I,me for First-Perspective novel
#text_resolved = text_resolved.replace("I", "Jane").replace("me", "Jane")

In [ ]:
print(len(JE_coref))
print(len(GE_coref))
print(len(PP_coref))
print(len(LO_coref))

1147644
1121000
829294
719102


### Name Entity Recognition

In [ ]:
test = nlp("He acknowledged Miss Bennet to be pretty; but Miss Bennet smiled too much.")
for token in test:
  print(token.text,token.dep_)

He nsubj
acknowledged ROOT
Miss compound
Bennet dobj
to aux
be xcomp
pretty acomp
; punct
but cc
Miss compound
Bennet nsubj
smiled conj
too advmod
much advmod
. punct


In [ ]:
# process text
JE_doc = nlp(JE_coref)
LO_doc = nlp(LO_coref)
PP_doc = nlp(PP_coref)
GE_doc = nlp(GE_coref)

In [ ]:
def name_freq(doc):
  # choose "PERSON" from entities
  names = []
  for ent in doc.ents:
    if ent.label_ == "PERSON":
      names.append(ent.text)
  # generate name list with frequency
  name_list = dict(Counter(names))
  name_freq = dict(sorted(name_list.items(), key=lambda item: item[1],reverse=True))
  return(name_freq)

In [ ]:
print(name_freq(JE_doc))

{'Rochester': 742, 'Jane': 334, 'Reed': 238, 'Fairfax': 218, 'Diana': 153, 'Bessie': 139, 'Mary': 137, 'John': 114, 'St. John': 86, 'Brocklehurst': 73, 'Hannah': 73, 'Rivers': 72, 'Jane Eyre': 65, 'Miss Temple': 54, 'Ingram': 47, 'Eyre': 44, 'Temple': 44, 'Project Gutenberg-tm': 43, 'Oliver': 41, 'Miss Ingram': 35, 'Sam': 33, 'John Reed': 32, 'Helen Burns': 32, 'Edward': 32, 'Scatcherd': 30, 'Lowood': 29, 'Poole': 28, 'Briggs': 27, "Helen Burns's": 25, 'Lloyd': 24, 'Miss Miller': 23, 'Dent': 22, 'Miss Smith': 21, 'Carter': 21, 'Louisa': 20, 'Helen': 18, 'Eshton': 18, 'Missis': 17, 'Janet': 17, 'Rowland': 16, 'Amy': 16, 'Mason': 16, 'Abbot': 15, 'Miller': 15, 'Burns': 15, 'Adèle': 15, 'Mary Ingram': 14, "Jane Eyre's": 13, 'Robert': 13, 'Louisa Eshton': 13, 'Juggernaut': 12, "St. John's": 12, 'Miss Oliver': 12, 'Project Gutenberg': 12, "John Reed's": 11, 'Mary Rivers': 11, 'Morton': 11, 'Charlotte Brontë': 10, '&c.': 10, 'Richard': 10, 'Jane Rochester': 10, 'Clara': 10, 'Richard Mason': 

In [ ]:
print(name_freq(LO_doc))

{'Lo': 430, 'Lolita': 140, 'Humbert': 113, 'Haze': 58, 'Jean': 43, 'Dick': 38, 'Humbert Humbert': 36, 'Mona': 31, 'Dolores Haze': 29, 'Gaston': 23, 'John': 22, 'Quilty': 21, 'Bill': 21, 'Rita': 20, 'McCoo': 19, 'Mary': 19, 'Annabel': 15, 'Valeria': 15, 'Monique': 14, 'Louise': 13, 'Marlene': 13, "Dolores Haze's": 13, 'Roy': 11, 'Parkington': 10, 'Hays': 10, 'Lester': 9, 'Leslie': 8, 'Charlotte': 8, 'Jean Farlow': 8, 'Phalen': 8, 'Potts': 8, 'Avis': 8, 'Valechka': 7, 'Phyllis': 7, 'Miss Opposite': 7, 'Tony': 7, "Humbert Humbert's": 6, 'Pisky': 6, 'Opposite': 6, 'mummy': 6, 'Linda Hall': 6, 'Eva Rosen': 6, 'Tristramson': 6, 'Wace': 6, 'Mary Lore': 6, 'Frank': 6, 'Sybil': 5, 'Clare': 5, 'Miss Phalen': 5, 'I.      ': 5, 'Chatfields': 5, 'Adam': 5, 'Holmes': 5, 'Holigan': 5, 'Diana': 5, 'Dorothy': 5, 'Herr Doktor': 4, 'Leslie Tomson': 4, 'God': 4, 'Charlotte Haze': 4, 'Humberts': 4, 'Hourglass Lake': 4, "Miss O.'s": 4, 'Charlie': 4, 'Mother': 4, 'Brown': 4, 'Swoon': 4, "Gaston Godin's": 4, 

In [ ]:
print(name_freq(PP_doc))

{'Elizabeth': 1255, 'Darcy': 977, 'Bennet': 579, 'Jane': 493, 'Bingley': 386, 'Collins': 321, 'Wickham': 203, 'Lady Catherine': 200, 'Gardiner': 166, 'Lizzy': 133, 'Kitty': 106, 'Lydia': 102, 'Miss Bingley': 80, 'Mary': 79, 'Forster': 65, 'Hurst': 62, 'Lucas': 55, 'Fitzwilliam': 47, 'Miss Bennet': 44, 'Austen': 43, 'Philips': 43, 'Project Gutenberg-tm': 42, 'William': 38, 'George Allen': 34, 'William Lucas': 34, 'Copyright 1894': 33, "Lady Catherine's": 32, 'Long': 28, 'Catherine': 23, 'Maria': 23, 'Miss de Bourgh': 22, 'Elizabeth Bennet': 22, 'Reynolds': 22, 'Lady Catherine de Bourgh': 21, 'Pemberley': 21, 'Netherfield': 21, 'Lady Lucas': 19, 'Rosings': 18, 'De Bourgh': 16, 'zabeth': 16, 'Louisa': 15, 'Kent': 15, 'Charles': 14, 'Brighton': 13, 'Project Gutenberg': 13, 'Longbourn': 12, 'F.': 12, 'Carter': 10, 'Denny': 10, 'lizabeth': 10, 'Jenkinson': 10, 'Jane Austen': 9, 'Caroline': 9, 'Harriet': 8, 'Hunsford': 7, "William Lucas's": 7, 'Meryton': 7, 'Jones': 7, 'Miss Bennets': 7, 'Pro

In [ ]:
print(name_freq(GE_doc))

{'Joe': 1186, 'Herbert': 562, 'Havisham': 484, 'Wemmick': 452, 'Jaggers': 396, 'Wopsle': 223, 'Pumblechook': 208, 'Estella': 207, 'Pocket': 192, 'Provis': 137, 'Biddy': 135, 'Drummle': 107, 'Orlick': 93, 'Miss Havisham': 64, 'Pip': 48, 'Trabb': 47, 'Skiffins': 44, 'Mum': 41, 'Sarah Pocket': 41, 'Joseph': 37, 'Project Gutenberg-tm': 34, 'Uncle Pumblechook': 32, 'Handel': 31, 'Gargery': 30, 'Magwitch': 30, "Havisham’s's": 26, 'Clara': 26, 'Mike': 25, 'Hubble': 21, 'Jack': 21, 'Matthew': 19, 'Matthew Pocket': 19, 'Flopson': 16, 'Spider': 16, 'Avenger': 16, 'Tom': 15, 'Joseph Gargery': 15, 'Coiler': 15, 'Compeyson': 15, 'Magwitch,—Provis': 15, 'Joe Gargery': 14, 'Herbert Pocket': 13, 'Camilla': 12, "Herbert Pocket's": 12, 'Abel Magwitch': 12, 'Project Gutenberg': 12, 'Hammersmith': 11, 'Startop': 11, 'William': 11, 'I.': 10, 'Jane': 10, 'sherry smelt': 9, 'Brandley': 9, 'Georgiana': 9, 'Charles Dickens': 8, 'Richard': 8, 'Dunstable': 8, 'Whimple': 8, 'Bentley Drummle': 8, 'Project Gutenber

### Dependence 

In [ ]:
subLabels = ["nsubj","nsubjpass","csubj","csubjpass","agent","expl"]
objLabels = ["pobj","dobj","dative","attr","oprd"]

def dependency(doc, female_char, male_char):
  '''
  input: doc (spacy.tokens.doc.Doc): text data
         char_name (list(str)): name list in text, several one-words for one character
  output: all_dep (dict(dict)):{char_name:{pos:times}}
          sub_obj (dict(dict)):{char_name:{"sub"/"obj":times}} 
          gender_diff (DataFrame):"male"/"female"*"sub"/"obj"/"ratio"
  '''
  all_dep = {}
  sub_obj = {}
  gender_diff = pd.DataFrame(data = [[0,0,1],[0,0,1]],columns = ["sub","obj","ratio"],index = ["female","male"])
  char_names = male_char+female_char

  for char_name in char_names:
    all_dep[char_name] = {}
  for char_name in char_names:
    sub_obj[char_name] = {}
  
  for token in doc:
    if token.text in all_dep:
      all_dep[token.text][token.dep_] = all_dep[token.text].get(token.dep_, 0) + 1

  for char_name in char_names:
    for dep in all_dep[char_name]:
      if dep in subLabels:
        sub_obj[char_name]['sub'] = sub_obj[char_name].get('sub', 0) + all_dep[char_name][dep]
      elif dep in objLabels:
        sub_obj[char_name]['obj'] = sub_obj[char_name].get('obj', 0) + all_dep[char_name][dep]
  
  for char_name in char_names:
    if char_name in male_char:
      gender_diff['sub']['male'] += sub_obj[char_name].get('sub', 0)
      gender_diff['obj']['male'] += sub_obj[char_name].get('obj', 0)
    else:
      gender_diff['sub']['female'] += sub_obj[char_name].get('sub', 0)
      gender_diff['obj']['female'] += sub_obj[char_name].get('obj', 0)
  gender_diff['ratio'] = gender_diff['sub']/gender_diff['obj']
  print(sub_obj)
  print(gender_diff)
  stat, pval = proportions_ztest(gender_diff['obj'], gender_diff['obj']+gender_diff['sub'])
  print("z-text: stat: {}, p-val: {}".format(stat,pval))


In [ ]:
## Denpendency parsing for Jane Eyre
JE_female = ['Jane', 'Eyre', 'Janet', 'Sarah', 'Reed', 'Gibson', 'Eliza', 'Georgiana', 'Bessie', 'Lee', 'Martha', 'Abbot', 'Maria', 'Temple', 'Scatcherd', 'Helen', 'Burns', 'Alice', 'Fairfax', 'Adèle', 'Varens', 'Grace', 'Poole', 'Leah', 'Bertha', 'Antoinetta', 'Mary', 'Diana', 'Rosamond', 'Sophie', 'Eshton', 'Miller', 'Hannah']
JE_male= ['John', 'Lloyd', 'Brocklehurst', 'Edward', 'Rochester', 'Blanche', 'Ingram', 'Richard', 'Mason', 'Robert', 'Leaven', 'John', 'Rivers', 'Briggs','Oliver']
dependency(JE_doc, JE_female, JE_male)

{'John': {'sub': 456, 'obj': 172}, 'Lloyd': {'obj': 11, 'sub': 9}, 'Brocklehurst': {'obj': 20, 'sub': 45}, 'Edward': {'obj': 5, 'sub': 21}, 'Rochester': {'sub': 374, 'obj': 190}, 'Blanche': {'sub': 18, 'obj': 10}, 'Ingram': {'sub': 131, 'obj': 59}, 'Richard': {'sub': 3, 'obj': 2}, 'Mason': {'obj': 42, 'sub': 48}, 'Robert': {'sub': 5, 'obj': 4}, 'Leaven': {'obj': 2, 'sub': 3}, 'Rivers': {'sub': 63, 'obj': 20}, 'Briggs': {'obj': 6, 'sub': 19}, 'Oliver': {'sub': 32, 'obj': 13}, 'Jane': {'obj': 74, 'sub': 164}, 'Eyre': {'obj': 53, 'sub': 55}, 'Janet': {'sub': 9, 'obj': 2}, 'Sarah': {'sub': 2}, 'Reed': {'sub': 157, 'obj': 78}, 'Gibson': {'obj': 1, 'sub': 1}, 'Eliza': {'obj': 16, 'sub': 40}, 'Georgiana': {'sub': 36, 'obj': 16}, 'Bessie': {'obj': 40, 'sub': 93}, 'Lee': {}, 'Martha': {}, 'Abbot': {'obj': 5, 'sub': 8}, 'Maria': {}, 'Temple': {'obj': 19, 'sub': 55}, 'Scatcherd': {'obj': 14, 'sub': 10}, 'Helen': {'sub': 35, 'obj': 28}, 'Burns': {'sub': 31, 'obj': 17}, 'Alice': {}, 'Fairfax': {'su

In [ ]:
## Denpendency parsing for Lolita
LO_male = ["Humbert","Clare","Quilty","John","Ivor","Ray","Charlie","Frederick","Beale","Gaston","Grodin","Windmuller","McCoo","Bill","Leslie","Phyllis","Frank","Tony"]
LO_female = ["Lolita","Haze","Dolores","Lo","Charlotte","Annabel","Leigh","Valeria","Jean","Dick","Schiller","Rita","Monique","Mona","Louise","Shirley","Holmes","Pratt","Barbara","Vivian","Darkbloom","Mary","Lore","Avis","Dolly","Holigan"]
dependency(LO_doc, LO_female, LO_male)

{'Humbert': {'sub': 69, 'obj': 37}, 'Clare': {'obj': 1, 'sub': 1}, 'Quilty': {'obj': 19, 'sub': 25}, 'John': {'sub': 15, 'obj': 5}, 'Ivor': {'obj': 2, 'sub': 2}, 'Ray': {'obj': 1}, 'Charlie': {'sub': 1, 'obj': 1}, 'Frederick': {}, 'Beale': {'obj': 2}, 'Gaston': {'obj': 9, 'sub': 12}, 'Grodin': {}, 'Windmuller': {'obj': 1, 'sub': 1}, 'McCoo': {'sub': 8, 'obj': 4}, 'Bill': {'obj': 5, 'sub': 7}, 'Leslie': {'sub': 5, 'obj': 2}, 'Phyllis': {'sub': 3, 'obj': 2}, 'Frank': {'obj': 2, 'sub': 3}, 'Tony': {'sub': 4}, 'Lolita': {'obj': 105, 'sub': 80}, 'Haze': {'obj': 51, 'sub': 89}, 'Dolores': {'obj': 15, 'sub': 13}, 'Lo': {'sub': 190, 'obj': 100}, 'Charlotte': {'obj': 48, 'sub': 57}, 'Annabel': {'obj': 11, 'sub': 8}, 'Leigh': {}, 'Valeria': {'obj': 12, 'sub': 11}, 'Jean': {'sub': 28, 'obj': 11}, 'Dick': {'sub': 19, 'obj': 9}, 'Schiller': {'sub': 4, 'obj': 6}, 'Rita': {'sub': 12, 'obj': 5}, 'Monique': {'sub': 4, 'obj': 7}, 'Mona': {'sub': 17, 'obj': 9}, 'Louise': {'sub': 4, 'obj': 3}, 'Shirley': 

In [ ]:
## Denpendency parsing for PP
PP_male = ["Darcy","Collins","Wickham","Fitzwilliam","Colonel","William","Carter","Denny","Charles","Jones"]
PP_female = ["Elizabeth","Jane","Bingley","Lizzy","Kitty","Lydia","Mary","Catherine","Bennet","Jenkinson","Maria","Bourgh","Reynolds","Caroline","King","Charlotte","Louisa","Hill","Eliza"]
dependency(PP_doc, PP_female, PP_male)

{'Darcy': {'sub': 439, 'obj': 299}, 'Collins': {'obj': 95, 'sub': 130}, 'Wickham': {'obj': 105, 'sub': 122}, 'Fitzwilliam': {'obj': 17, 'sub': 26}, 'Colonel': {}, 'William': {'sub': 18, 'obj': 10}, 'Carter': {'obj': 3, 'sub': 3}, 'Denny': {'sub': 8, 'obj': 2}, 'Charles': {'sub': 5, 'obj': 3}, 'Jones': {'obj': 2, 'sub': 3}, 'Elizabeth': {'sub': 677, 'obj': 239}, 'Jane': {'obj': 168, 'sub': 215}, 'Bingley': {'sub': 293, 'obj': 183}, 'Lizzy': {'obj': 23, 'sub': 72}, 'Kitty': {'obj': 32, 'sub': 51}, 'Lydia': {'obj': 71, 'sub': 176}, 'Mary': {'sub': 43, 'obj': 26}, 'Catherine': {'obj': 65, 'sub': 103}, 'Bennet': {'obj': 162, 'sub': 340}, 'Jenkinson': {'sub': 3, 'obj': 4}, 'Maria': {'obj': 5, 'sub': 7}, 'Bourgh': {'obj': 33, 'sub': 29}, 'Reynolds': {'sub': 11, 'obj': 3}, 'Caroline': {'sub': 7, 'obj': 9}, 'King': {'obj': 6, 'sub': 5}, 'Charlotte': {'obj': 36, 'sub': 58}, 'Louisa': {'obj': 3, 'sub': 9}, 'Hill': {'obj': 3, 'sub': 4}, 'Eliza': {'obj': 1, 'sub': 7}}
         sub   obj     ratio
f

In [ ]:
## Denpendency parsing for GE
GE_male = ["Joe", "Herbert", "Wemmick","Jaggers","Pip","Pumblechook","Wopsle","Provis","Drummle","Orlick","Handel","Joseph","Jack","Trabb","Pumblechook","Gargery","Matthew","Magwitch","Mike","Avenger","Richard","Tom","John","Waldengarver","Compeyson","Spider","Bill","William", "Startop","Brandley","Arthur","Raymond", "Barley","Barnard","Campbell"]
GE_female = ["Havisham", "Estella","Biddy","Skiffins","Sarah","Clara","Jane","Camilla","Mum", "Hubble","Flopson","Jane","Coiler", "Whimple"]
dependency(GE_doc, GE_female, GE_male)

{'Joe': {'obj': 253, 'sub': 555}, 'Herbert': {'obj': 154, 'sub': 314}, 'Wemmick': {'obj': 121, 'sub': 258}, 'Jaggers': {'obj': 86, 'sub': 222}, 'Pip': {'obj': 64, 'sub': 112}, 'Pumblechook': {'obj': 150, 'sub': 296}, 'Wopsle': {'sub': 110, 'obj': 44}, 'Provis': {'obj': 45, 'sub': 57}, 'Drummle': {'obj': 65, 'sub': 100}, 'Orlick': {'obj': 34, 'sub': 53}, 'Handel': {'obj': 11, 'sub': 23}, 'Joseph': {'sub': 14, 'obj': 15}, 'Jack': {'sub': 6, 'obj': 5}, 'Trabb': {'obj': 17, 'sub': 19}, 'Gargery': {'obj': 16, 'sub': 31}, 'Matthew': {'obj': 2, 'sub': 5}, 'Magwitch': {'obj': 32, 'sub': 11}, 'Mike': {'sub': 16, 'obj': 4}, 'Avenger': {'obj': 9, 'sub': 2}, 'Richard': {'obj': 2, 'sub': 1}, 'Tom': {'sub': 3, 'obj': 7}, 'John': {'sub': 2, 'obj': 2}, 'Waldengarver': {'sub': 1, 'obj': 2}, 'Compeyson': {'obj': 24, 'sub': 45}, 'Spider': {'sub': 6, 'obj': 10}, 'Bill': {'obj': 2, 'sub': 2}, 'William': {'sub': 7}, 'Startop': {'sub': 15, 'obj': 11}, 'Brandley': {'sub': 3, 'obj': 1}, 'Arthur': {'obj': 2}, '